In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import pickle

# Reading a local database
**Can be Applied to any similar databse**

### Here, transforming B to Billions, M to Millions, T to Trillions, % to percentage, N/A to Nan

In [8]:
filename='Dict_Fundamentals.csv'
with open(filename, 'rb') as handle:
  data = handle.read()
results = pickle.loads(data)
##
df=results['dfs'][0]
for i in range(1,len(results['dfs'])):
    try:
        df=pd.concat([df,
            results['dfs'][i]],
            join='outer',axis=0)
    except:
        pass

In [15]:
df.head()

Asset,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta (5Y Monthly),...,Diluted EPS,Quarterly Earnings Growth,Total Cash,Total Cash Per Share,Total Debt,Total Debt/Equity,Current Ratio,Book Value Per Share,Operating Cash Flow,Levered Free Cash Flow
AAPL,2.17T,2.19T,29.16,24.43,1.46,6.65,31.29,6.73,21.93,1.20,...,4.45,110.10%,69.83B,4.18,121.64B,175.84,1.14,4.15,99.59B,80.12B
TSLA,639.21B,643.15B,664.87,107.72,3.71,17.79,27.76,17.90,141.32,1.98,...,1.00,2,17.14B,17.79,12.51B,51.14,1.66,23.90,8.02B,3.54B
WMT,396.84B,448.65B,29.69,24.11,4.07,0.71,4.92,0.80,11.78,0.47,...,4.75,N/A,17.74B,6.3,63.7B,72.78,0.97,28.69,36.07B,15.27B
AMZN,1.67T,1.68T,62.90,45.95,1.59,3.98,16.13,4.00,29.34,1.15,...,52.56,219.80%,73.27B,145.28,101.52B,98.26,1.05,205.00,67.21B,31.39B
HL,3.9B,3.75B,N/A,24.30,-0.95,5.64,2.28,5.42,16.88,2.29,...,-0.03,N/A,129.83M,0.24,533.65M,31.35,1.90,3.20,180.79M,84.55M


### Data Cleaning

In [17]:
for j,k in zip(['N/A','nan'],[np.nan,np.nan]):
    df.replace(j,k,inplace=True)
##
for row in range(len(df)):
    for i in range(len(df.values[row])):
        try:
            val=df.values[row][i]
            if val.endswith('M'): #Million
                df.replace(val,1e6*float(val[:-1]),inplace=True)
            if val.endswith('B'): #Billion
                df.replace(val,1e9*float(val[:-1]),inplace=True)
            if val.endswith('T'): #Trillion
                df.replace(val,1e12*float(val[:-1]),inplace=True)
            if val.endswith('%'):
                df.replace(val,1e-2*float(val[:-1]),inplace=True)
        except:
            pass

### final database

In [18]:
df

Asset,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta (5Y Monthly),...,Diluted EPS,Quarterly Earnings Growth,Total Cash,Total Cash Per Share,Total Debt,Total Debt/Equity,Current Ratio,Book Value Per Share,Operating Cash Flow,Levered Free Cash Flow
AAPL,2.170000e+12,2.190000e+12,29.16,24.43,1.46,6.65,31.29,6.73,21.93,1.20,...,4.45,1.101,6.983000e+10,4.18,1.2164e+11,175.84,1.14,4.15,9.959000e+10,8.012000e+10
TSLA,6.392100e+11,6.431500e+11,664.87,107.72,3.71,17.79,27.76,17.90,141.32,1.98,...,1.00,2,1.714000e+10,17.79,1.251e+10,51.14,1.66,23.90,8.020000e+09,3.540000e+09
WMT,3.968400e+11,4.486500e+11,29.69,24.11,4.07,0.71,4.92,0.80,11.78,0.47,...,4.75,NaN,1.774000e+10,6.3,6.37e+10,72.78,0.97,28.69,3.607000e+10,1.527000e+10
AMZN,1.670000e+12,1.680000e+12,62.90,45.95,1.59,3.98,16.13,4.00,29.34,1.15,...,52.56,2.198,7.327000e+10,145.28,1.0152e+11,98.26,1.05,205.00,6.721000e+10,3.139000e+10
HL,3.900000e+09,3.750000e+09,NaN,24.30,-0.95,5.64,2.28,5.42,16.88,2.29,...,-0.03,NaN,1.298300e+08,0.24,5.3365e+08,31.35,1.90,3.20,1.807900e+08,8.455000e+07
AG,4.000000e+09,3.220000e+09,147.50,227.57,1.96,10.98,4.16,8.86,29.09,0.90,...,0.11,NaN,2.749000e+08,1.24,1.7328e+08,20.38,3.50,3.83,7.971000e+07,-1.029000e+07
AUY,4.810000e+09,5.820000e+09,22.77,26.37,0.62,2.96,1.09,3.58,6.88,1.46,...,0.22,0.216,6.984000e+08,0.72,1.06e+09,20.16,1.56,4.59,6.486000e+08,4.045100e+08
MAG,1.850000e+09,1.660000e+09,NaN,-278.57,NaN,NaN,5.84,NaN,-198.57,0.96,...,-0.08,NaN,9.401000e+07,0.99,476k,0.15,105.90,3.34,-6.210000e+06,-7.980000e+06
AEM,1.655000e+10,1.776000e+10,24.92,21.41,0.60,4.87,2.92,5.22,9.96,0.77,...,2.74,NaN,1.319500e+08,0.54,1.68e+09,29.58,2.30,23.41,1.390000e+09,4.769200e+08
AU,9.340000e+09,9.780000e+09,9.88,9.07,NaN,2.11,2.53,2.21,4.97,0.23,...,2.27,NaN,1.330000e+09,3.19,2.08e+09,55.72,2.43,8.86,1.650000e+09,1.060000e+09
